In [1]:
# Cell #: 1 Library Imports
import pandas as pd
import networkx as nx
import uunet.multinet as ml
import itertools
import numpy as np  
from math import log

In [2]:
#Cell #: 2 File Paths
# File paths for the Excel sheets
file_paths = [
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_1.xlsx",
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_2.xlsx",
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_3.xlsx",
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_4.xlsx",
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_5.xlsx"
]


In [3]:
# Cell #: 3 Network Triming Logic
# Reading the Excel files
cases_data = [pd.read_excel(file_path) for file_path in file_paths]

# Function to apply the filtering logic and track removed edges
def filter_edges_and_save_removed(cases):
    max_flow_per_edge = {}
    filtered_cases = []
    removed_edges = []

    # Step 1: Determine the maximum flow for each edge across all cases
    for case in cases:
        for _, row in case.iterrows():
            edge = (row['From_Node'], row['To_Node'])
            flow = row['Flow']
            max_flow_per_edge[edge] = max(max_flow_per_edge.get(edge, 0), flow)

    # Steps 2 and 3: Filtering edges and tracking removed edges in each case
    for case in cases:
        filtered_rows = []
        removed_edges_rows = []
        for _, row in case.iterrows():
            edge = (row['From_Node'], row['To_Node'])
            if row['Flow'] >= max_flow_per_edge[edge] * 0.9:
                filtered_rows.append(row)
            else:
                removed_edges_rows.append(row)
        filtered_cases.append(pd.DataFrame(filtered_rows))
        removed_edges.append(pd.DataFrame(removed_edges_rows))

    return filtered_cases, removed_edges

# Apply the filtering logic and get removed edges
filtered_cases_data, removed_edges_data = filter_edges_and_save_removed(cases_data)

# Print the results and optionally save them to files
for i, (filtered_case, removed_edges_case) in enumerate(zip(filtered_cases_data, removed_edges_data), 1):
    #print(f"Case {i} - Filtered Data:\n", filtered_case.head())
    print(f"Case {i} - Removed Edges:\n", removed_edges_case, "\n")
    # Optionally, save the filtered cases to files
    filtered_case.to_excel(f"/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Filtered_Case_{i}.xlsx", index=False)
    
    # Optionally, save the removed edges to files
    #removed_edges_case.to_excel(f"/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Removed_Edges_Case_{i}.xlsx", index=False)


Case 1 - Removed Edges:
      From_Node  From_Layer  To_Node  To_Layer        Flow
11         2.0         1.0     12.0       1.0   39.476633
16        59.0         1.0     70.0       1.0   65.473726
17        12.0         1.0     30.0       1.0   77.406378
18        44.0         1.0     30.0       1.0  102.065698
19        18.0         1.0     44.0       1.0  111.489735
..         ...         ...      ...       ...         ...
241      148.0         2.0    159.0       2.0  113.276302
242      159.0         2.0    161.0       2.0  113.276302
244      125.0         2.0    124.0       2.0  114.968040
246      124.0         2.0    136.0       2.0  129.942968
247      136.0         2.0    148.0       2.0  129.942968

[66 rows x 5 columns] 

Case 2 - Removed Edges:
      From_Node  From_Layer  To_Node  To_Layer       Flow
0          3.0         1.0     34.0       1.0   0.000000
1          3.0         1.0     34.0       1.0   0.000000
2          6.0         1.0     36.0       1.0   0.000000
3

In [ ]:
#Cell #: 4 Multiplex Network Construction Logic
print("Initializing multiplex network...")
multiplex_net = ml.empty()

# A variable to keep track of the layer index
layer_index = 1

# Iterate over the filtered data and add each as a layer to the multiplex network
for df in filtered_cases_data:
    unique_layer_name = f"Layer_{layer_index}"
    print(f"\nAdding layer: {unique_layer_name}")

    # Debug: Print a sample of the DataFrame
    print(f"DataFrame sample from {unique_layer_name}:\n", df.head())

    # Prepare vertices and add them to the network
    vertices = {'actor': list(set(df['From_Node']).union(df['To_Node'])), 
                'layer': [unique_layer_name] * len(set(df['From_Node']).union(df['To_Node']))}
    print(f"Adding vertices to layer {unique_layer_name}: {vertices['actor']}")
    ml.add_vertices(multiplex_net, vertices)

    # Prepare edges and add them to the network
    edges = {
        'from_actor': df['From_Node'].tolist(),
        'from_layer': [unique_layer_name] * len(df),
        'to_actor': df['To_Node'].tolist(),
        'to_layer': [unique_layer_name] * len(df)
    }
    print(f"Adding {len(df)} edges to layer {unique_layer_name}")
    print(f"Sample edges for layer {unique_layer_name}: {edges['from_actor'][:5]} -> {edges['to_actor'][:5]}")
    ml.add_edges(multiplex_net, edges)

    layer_index += 1

print("\nNetwork construction completed.")
print(f"Total layers in network: {len(ml.layers(multiplex_net))}")

print("\nNetwork summary:")
print(ml.summary(multiplex_net))


In [5]:
# Cell #: 5 Finding the Exclusive Neighbors
# Retrieve the exclusive neighbors of all actors in all Layers
all_layers = ml.layers(multiplex_net)

exclusive_neighbors = {}

for layer in all_layers:
    print(f"\nFinding exclusive neighbors for layer {layer}")

    # Retrieve all vertices in the current layer
    vertices_info = ml.vertices(multiplex_net, [layer])
    actors = vertices_info['actor']  # List of actor IDs in the layer

    # Prepare to store exclusive neighbors for the current layer
    exclusive_neighbors[layer] = {}

    # Find exclusive neighbors for each actor in the layer
    for actor_id in actors:
        # Retrieve exclusive neighbors for the actor
        neighbors = ml.xneighbors(multiplex_net, actor_id, layers=[layer], mode='all')
        #print(f"Type of neighbors for actor {actor_id} in layer {layer}: {type(neighbors)}")
        exclusive_neighbors[layer][actor_id] = neighbors


# Commented out file saving part
# file_path = '/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/exclusive_neighbors_Indivisual_layers70.txt'
# with open(file_path, 'w') as file:
#     for layer, neighbors_dict in exclusive_neighbors.items():
#         file.write(f"Layer {layer}:\n")
#         for actor_id, neighbors in neighbors_dict.items():
#             if neighbors:  # Only write if the actor has exclusive neighbors
#                 file.write(f"Actor {actor_id}: Exclusive neighbors: {neighbors}\n")
#         file.write("\n")

# Display the exclusive neighbors data
for layer, neighbors_dict in exclusive_neighbors.items():
    print(f"Layer {layer}:")
    for actor_id, neighbors in neighbors_dict.items():
        if neighbors:  # Only display if the actor has exclusive neighbors
            print(f"Actor {actor_id}: Exclusive neighbors: {neighbors}")
    print("\n")



Finding exclusive neighbors for layer Layer_1

Finding exclusive neighbors for layer Layer_4

Finding exclusive neighbors for layer Layer_5

Finding exclusive neighbors for layer Layer_2

Finding exclusive neighbors for layer Layer_3
Layer Layer_1:
Actor 2.0: Exclusive neighbors: {'6.0'}
Actor 6.0: Exclusive neighbors: {'36.0', '2.0', '11.0'}
Actor 11.0: Exclusive neighbors: {'6.0'}
Actor 13.0: Exclusive neighbors: {'30.0'}
Actor 23.0: Exclusive neighbors: {'34.0'}
Actor 30.0: Exclusive neighbors: {'13.0'}
Actor 34.0: Exclusive neighbors: {'23.0'}
Actor 36.0: Exclusive neighbors: {'6.0'}
Actor 160.0: Exclusive neighbors: {'174.0', '176.0'}
Actor 174.0: Exclusive neighbors: {'160.0'}
Actor 175.0: Exclusive neighbors: {'191.0'}
Actor 176.0: Exclusive neighbors: {'160.0'}
Actor 191.0: Exclusive neighbors: {'175.0'}


Layer Layer_4:


Layer Layer_5:
Actor 133.0: Exclusive neighbors: {'130.0'}
Actor 11.0: Exclusive neighbors: {'37.0', '62.0'}
Actor 15.0: Exclusive neighbors: {'43.0'}
Actor

In [6]:
# Cell #: 6 Link Prediction : Through Modified Jaccard Coefficient
# Convert the multiplex network into NetworkX graphs for each layer
nx_graphs = ml.to_nx_dict(multiplex_net)

# Define the modified Jaccard coefficient function
def modified_jaccard_coefficient(G, exclusive_neighbors):
    def predict(u, v):
        neighbors_u = exclusive_neighbors.get(u, set())
        neighbors_v = exclusive_neighbors.get(v, set())
        union_size = len(neighbors_u | neighbors_v)
        if union_size == 0:
            return 0
        intersection_size = len(neighbors_u & neighbors_v)
        return intersection_size / union_size

    return ((u, v, predict(u, v)) for u, v in nx.non_edges(G))

# Calculate and store Jaccard scores
jaccard_scores = {}

for layer, layer_exclusive_neighbors in exclusive_neighbors.items():
    if layer_exclusive_neighbors:  # Check if the layer has exclusive neighbors
        G = nx_graphs[layer]
        jaccard_scores[layer] = {}
        for u, v, score in modified_jaccard_coefficient(G, layer_exclusive_neighbors):
            if score > 0:
                jaccard_scores[layer][f"{u} - {v}"] = score
                print(f"Link prediction for {layer}: {u} - {v}: {score}")
    else:
        print(f"No exclusive neighbors for {layer}, skipping link prediction.")


Link prediction for Layer_1: 176.0 - 174.0: 1.0
Link prediction for Layer_1: 2.0 - 36.0: 1.0
Link prediction for Layer_1: 2.0 - 11.0: 1.0
Link prediction for Layer_1: 36.0 - 11.0: 1.0
Link prediction for Layer_5: 17.0 - 28.0: 0.25
Link prediction for Layer_5: 33.0 - 11.0: 0.5
Link prediction for Layer_5: 19.0 - 28.0: 0.3333333333333333
Link prediction for Layer_5: 37.0 - 62.0: 0.3333333333333333
Link prediction for Layer_5: 75.0 - 39.0: 0.5
Link prediction for Layer_5: 133.0 - 141.0: 1.0
Link prediction for Layer_5: 62.0 - 39.0: 0.3333333333333333
Link prediction for Layer_5: 62.0 - 20.0: 0.3333333333333333
Link prediction for Layer_5: 39.0 - 20.0: 0.3333333333333333
Link prediction for Layer_5: 11.0 - 28.0: 0.25


In [7]:
# Cell #: 7 Link Prediction : Through Modified Adamic-Adar Index

# Convert the multiplex network into NetworkX graphs for each layer
nx_graphs = ml.to_nx_dict(multiplex_net)

# Define the modified Adamic-Adar index function
def modified_adamic_adar_index(G, exclusive_neighbors):
    def predict(u, v):
        common_neighbors = exclusive_neighbors.get(u, set()) & exclusive_neighbors.get(v, set())
        return sum(1 / log(len(exclusive_neighbors.get(w, set()))) for w in common_neighbors)

    return ((u, v, predict(u, v)) for u, v in nx.non_edges(G))

# Calculate and store Adamic-Adar scores
adamic_adar_scores = {}

for layer, layer_exclusive_neighbors in exclusive_neighbors.items():
    if layer_exclusive_neighbors:
        G = nx_graphs[layer]
        adamic_adar_scores[layer] = {}
        for u, v, score in modified_adamic_adar_index(G, layer_exclusive_neighbors):
            if score > 0:
                adamic_adar_scores[layer][f"{u} - {v}"] = score
                print(f"Adamic-Adar prediction for {layer}: {u} - {v}: {score}")
    else:
        print(f"No exclusive neighbors for {layer}, skipping Adamic-Adar prediction.")


Adamic-Adar prediction for Layer_1: 176.0 - 174.0: 1.4426950408889634
Adamic-Adar prediction for Layer_1: 2.0 - 36.0: 0.9102392266268373
Adamic-Adar prediction for Layer_1: 2.0 - 11.0: 0.9102392266268373
Adamic-Adar prediction for Layer_1: 36.0 - 11.0: 0.9102392266268373
Adamic-Adar prediction for Layer_5: 17.0 - 28.0: 1.4426950408889634
Adamic-Adar prediction for Layer_5: 33.0 - 11.0: 1.4426950408889634
Adamic-Adar prediction for Layer_5: 19.0 - 28.0: 1.4426950408889634
Adamic-Adar prediction for Layer_5: 37.0 - 62.0: 1.4426950408889634
Adamic-Adar prediction for Layer_5: 75.0 - 39.0: 1.4426950408889634
Adamic-Adar prediction for Layer_5: 133.0 - 141.0: 1.4426950408889634
Adamic-Adar prediction for Layer_5: 62.0 - 39.0: 0.9102392266268373
Adamic-Adar prediction for Layer_5: 62.0 - 20.0: 0.9102392266268373
Adamic-Adar prediction for Layer_5: 39.0 - 20.0: 0.9102392266268373
Adamic-Adar prediction for Layer_5: 11.0 - 28.0: 1.4426950408889634


In [20]:
# Cell #: 8 Results Normalization Logic

def normalize_by_max(scores):
    # Flatten all scores into a single list, excluding empty layers
    all_scores = [score for layer_scores in scores.values() if layer_scores for score in layer_scores.values()]
    
    # Check if there are no scores to normalize
    if not all_scores:
        print("No scores to normalize.")
        return {}
    
    # Find the maximum score for normalization
    max_score = max(all_scores)
    
    # Normalize the scores
    normalized = {}
    for layer, layer_scores in scores.items():
        if layer_scores:  # Check if the layer has scores
            normalized[layer] = {link: score / max_score for link, score in layer_scores.items()}
    
    return normalized

# Normalize Jaccard and Adamic-Adar scores
normalized_jaccard = normalize_by_max(jaccard_scores)
normalized_adamic_adar = normalize_by_max(adamic_adar_scores)

# Debugging: Display all normalized scores
print("All Normalized Jaccard Scores:")
for layer, links in normalized_jaccard.items():
    print(f"Layer {layer}:")
    for link, score in links.items():
        print(f"{link}: {score}")
    print()

print("All Normalized Adamic-Adar Scores:")
for layer, links in normalized_adamic_adar.items():
    print(f"Layer {layer}:")
    for link, score in links.items():
        print(f"{link}: {score}")
    print()

All Normalized Jaccard Scores:
Layer Layer_1:
176.0 - 174.0: 1.0
2.0 - 36.0: 1.0
2.0 - 11.0: 1.0
36.0 - 11.0: 1.0

Layer Layer_5:
17.0 - 28.0: 0.25
33.0 - 11.0: 0.5
19.0 - 28.0: 0.3333333333333333
37.0 - 62.0: 0.3333333333333333
75.0 - 39.0: 0.5
133.0 - 141.0: 1.0
62.0 - 39.0: 0.3333333333333333
62.0 - 20.0: 0.3333333333333333
39.0 - 20.0: 0.3333333333333333
11.0 - 28.0: 0.25

All Normalized Adamic-Adar Scores:
Layer Layer_1:
176.0 - 174.0: 1.0
2.0 - 36.0: 0.6309297535714574
2.0 - 11.0: 0.6309297535714574
36.0 - 11.0: 0.6309297535714574

Layer Layer_5:
17.0 - 28.0: 1.0
33.0 - 11.0: 1.0
19.0 - 28.0: 1.0
37.0 - 62.0: 1.0
75.0 - 39.0: 1.0
133.0 - 141.0: 1.0
62.0 - 39.0: 0.6309297535714574
62.0 - 20.0: 0.6309297535714574
39.0 - 20.0: 0.6309297535714574
11.0 - 28.0: 1.0



In [14]:
# Cell #: 9 Dictionary to DataFrame Conversion Logic (Flow Prediction)
def scores_to_dataframe(normalized_scores):
    links = []
    for layer, layer_links in normalized_scores.items():
        for link, score in layer_links.items():
            node_u, node_v = map(float, link.split(" - "))
            links.append({'node_u': node_u, 'node_v': node_v, 'probability': score, 'layer': layer})
    return pd.DataFrame(links)

# Convert normalized scores to DataFrames
predicted_links_jaccard = scores_to_dataframe(normalized_jaccard)
predicted_links_adamic_adar = scores_to_dataframe(normalized_adamic_adar)

# Print the DataFrames
print("Predicted Links with Normalized Jaccard Scores:")
print(predicted_links_jaccard)  # Print the first few rows

print("\nPredicted Links with Normalized Adamic-Adar Scores:")
print(predicted_links_adamic_adar)  # Print the first few rows


Predicted Links with Normalized Jaccard Scores:
    node_u  node_v  probability    layer
0    176.0   174.0     1.000000  Layer_1
1      2.0    36.0     1.000000  Layer_1
2      2.0    11.0     1.000000  Layer_1
3     36.0    11.0     1.000000  Layer_1
4     17.0    28.0     0.250000  Layer_5
5     33.0    11.0     0.500000  Layer_5
6     19.0    28.0     0.333333  Layer_5
7     37.0    62.0     0.333333  Layer_5
8     75.0    39.0     0.500000  Layer_5
9    133.0   141.0     1.000000  Layer_5
10    62.0    39.0     0.333333  Layer_5
11    62.0    20.0     0.333333  Layer_5
12    39.0    20.0     0.333333  Layer_5
13    11.0    28.0     0.250000  Layer_5

Predicted Links with Normalized Adamic-Adar Scores:
    node_u  node_v  probability    layer
0    176.0   174.0      1.00000  Layer_1
1      2.0    36.0      0.63093  Layer_1
2      2.0    11.0      0.63093  Layer_1
3     36.0    11.0      0.63093  Layer_1
4     17.0    28.0      1.00000  Layer_5
5     33.0    11.0      1.00000  Layer

In [ ]:
# Cell #: 10 Calculating the Edge Weights (Flow Prediction)
def calculate_average_flow(node, exclusive_neighbors, layer_data):
    total_flow = 0
    node_str = str(node)  # Convert node ID to string for key lookup

    if node_str not in exclusive_neighbors:
        return 0  # Return 0 if node is not found

    neighbor_count = len(exclusive_neighbors[node_str])
    for neighbor_str in exclusive_neighbors[node_str]:
        neighbor = float(neighbor_str)  # Convert neighbor ID back to float for comparison
        flow = layer_data[((layer_data['From_Node'] == node) & (layer_data['To_Node'] == neighbor)) |
                          ((layer_data['From_Node'] == neighbor) & (layer_data['To_Node'] == node))]
        if not flow.empty:
            total_flow += flow.iloc[0]['Flow']

    return total_flow / neighbor_count if neighbor_count > 0 else 0

def calculate_edge_weight(row, exclusive_neighbors, filtered_cases_data):
    layer = row['layer']
    layer_data = filtered_cases_data[int(layer.split('_')[1]) - 1]

    avg_flow_u = calculate_average_flow(row['node_u'], exclusive_neighbors[layer], layer_data)
    avg_flow_v = calculate_average_flow(row['node_v'], exclusive_neighbors[layer], layer_data)

    final_weight = (avg_flow_u + avg_flow_v) / 2 * row['probability']
    return final_weight

# Apply weight calculation to DataFrames
predicted_links_jaccard['weight'] = predicted_links_jaccard.apply(lambda row: calculate_edge_weight(row, exclusive_neighbors, filtered_cases_data), axis=1)
predicted_links_adamic_adar['weight'] = predicted_links_adamic_adar.apply(lambda row: calculate_edge_weight(row, exclusive_neighbors, filtered_cases_data), axis=1)

# Filter and Print the DataFrames for links with probability >= 0.5
print("Predicted Links with Jaccard Scores and Weights (Probability >= 0.5):")
print(predicted_links_jaccard[predicted_links_jaccard['probability'] >= 0.5])

print("\nPredicted Links with Adamic-Adar Scores and Weights (Probability >= 0.5):")
print(predicted_links_adamic_adar[predicted_links_adamic_adar['probability'] >= 0.5])


In [ ]:
#### Cell to save the removed edges to a file ####
# Path to save the removed edges file
removed_edges_file_path = "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/removed_edges.txt"

# Save the removed edges to the file
with open(removed_edges_file_path, 'w') as file:
    for i, removed_edges_case in enumerate(removed_edges_data, 1):
        # Write the layer information
        file.write(f"Case {i} - Removed Edges:\n")
        
        # Write the removed edges
        for index, row in removed_edges_case.iterrows():
            file.write(f"{row['From_Node']} -> {row['To_Node']} (Flow: {row['Flow']})\n")
        
        # Write a separator between cases
        file.write("\n")

print(f"Removed edges saved to {removed_edges_file_path}")


In [ ]:

#### Debuging Cell ####
# Extract all layers from the multiplex network
all_layers = ml.layers(multiplex_net)

# Check if Layer 1 is present in the network
if 'Layer_1' in all_layers:
    # Retrieve all actors (nodes) in Layer 1
    actors_layer_1 = ml.actors(multiplex_net, layers=['Layer_1'])

    # Retrieve all edges (connections) in Layer 1
    edges_layer_1 = ml.edges(multiplex_net, layers1=['Layer_1'], layers2=['Layer_1'])

    # Format the edges data correctly using the provided keys
    edges_data = list(zip(edges_layer_1['from_actor'], edges_layer_1['to_actor']))

    # Print the formatted edges data
    print("Edges in Layer 1:", edges_data)
else:
    print("Layer_1 not found in the network.")
